In [2]:
import numpy as np
import subprocess
import zipfile
import os
import os
import getpass
import subprocess

# Download data by year

In [6]:
years = np.linspace(1991,2023,33)

for i in years:
    req_test = ['sbatch',
         '-J',
         '{}_era5_dl'.format(str(int(i))),
         '-e',
         '/share/temp/sgeoutput/nhashmeh/errors/%x.e%j',
         '-o',
         '/share/temp/sgeoutput/nhashmeh/output/%x.o%j',
         '-A',
         'proj_erf',
         '--mem=20G', '-c', '1', '-t', '24:00:00', '-p', 'all.q', '--wrap',
        '/ihme/homes/nhashmeh/miniconda3/envs/earth-analytics-python/bin/python /mnt/share/code/nhashmeh/downscaling/temperature/era5_data/cdsapi_run.py ' + str(int(i))]
    print("Requesting download for year: " + str(i))
    print(req_test)
    subprocess.call(req_test)

Requesting download for year: 1990.0
['sbatch', '-J', '1990_era5_dl', '-e', '/share/temp/sgeoutput/nhashmeh/errors/%x.e%j', '-o', '/share/temp/sgeoutput/nhashmeh/output/%x.o%j', '-A', 'proj_erf', '--mem=20G', '-c', '1', '-t', '24:00:00', '-p', 'all.q', '--wrap', '/ihme/homes/nhashmeh/miniconda3/envs/earth-analytics-python/bin/python /ihme/homes/nhashmeh/downscale_temperature/era5_data/cdsapi_run.py 1990']
Submitted batch job 29961414


# Unzip

In [ ]:
path_to_zip_file = '/mnt/share/erf/ERA5/three_hourly_temp_9km/'

for file in os.listdir(path_to_zip_file):
    if file.endswith('.zip'):
        print(file)
        with zipfile.ZipFile(path_to_zip_file + file, 'r') as zip_ref:
            print(file.split("_")[0])
            zip_ref.getinfo('data.nc').filename = str(file.split("_")[0]) + '_era5_temp.nc'
            print(zip_ref.getinfo('data.nc').filename)
            zip_ref.extract('data.nc', path = path_to_zip_file + 'downloaded_unzipped/')
    else:
        pass

# Hourly to daily

In [ ]:
path_to_files = '/mnt/share/erf/ERA5/three_hourly_temp_9km/downloaded_unzipped/'

for file in os.listdir(path_to_files):

    year = file.split('.')[0] # get year

    asdf = ['sbatch', '-J', '{}'.format(year), '-e', '/share/temp/sgeoutput/{}/errors/%x.e%j'.format(getpass.getuser()),
        '-o', '/share/temp/sgeoutput/{}/output/%x.o%j'.format(getpass.getuser()), '-A', 'proj_diet',
        '--mem=500G', '-c', '5', '-t', '240', '-C', 'archive', '-p', 'all.q', #'--wait',
        '--wrap', '/ihme/homes/nhashmeh/miniconda3/envs/earth-analytics-python/bin/python ' + '/mnt/share/code/nhashmeh/downscaling/temperature/era5_data/era5_data_hourly_to_daily.py ' + file]
    subprocess.call(asdf)
    # print(asdf)